# Natluk Community dashboard

1. From the menu, Run~"Run all cells".
2. Go to the bottom of the notebook to view the dashboard.

Introduction:
* Dashboard for Natluk inside Jupyterlite. 
* JupyterLite is a JupyterLab distribution that runs entirely in the browser built from the ground-up using JupyterLab widgets, components and extensions.
* Python code is compiled to webassembly using Pyodide. 
* This enables Python generated interactive dashboards to be integrated into websites without requiring a live Python kernel

### Install required packages

In [1]:
import piplite
await piplite.install("ipyvuetify")
await piplite.install("ipywidgets")
await piplite.install("ipyflex")
await piplite.install("plotly")

In [2]:
import os
import ipyvuetify as v
import ipywidgets as ipw
import ipyflex
import pandas as pd
import pyodide
import plotly
import plotly.express as px
import plotly.graph_objects as go
import json
import datetime as dt


/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
voila = os.getenv("VOILA_KERNEL_ID")
if voila is not None:
    editable = True
#     height = "calc(100vh - 60px)"
    height = "2500px"
else:
    editable = True
    height = "2500px"

### Get Natluk related data

In [4]:
# from transform.data_processing import get_all_addresses, read_data, read_global_data

LP_METHODS = [
    'Add Liquidity ETH',
    'Remove Liquidity ETH With Permit Supporting Fee On Transfer Tokens',
    'Remove Liquidity With Permit',
    'Remove Liquidity'
]
FARMING_METHODS = ['Stake', 'Unstake']
BUY_SELL_METHODS = [
    'Swap Exact ETH For Tokens',
    'Swap Exact Tokens For ETH Supporting Fee On Transfer Tokens',
    'Swap Exact Tokens For Tokens',
    'Swap Exact Tokens For Tokens Supporting Fee On Transfer Tokens',
    'Swap ETH For Exact Tokens', 'Swap'
]

NC_PATH = "https://raw.githubusercontent.com/DougRzz/nc-dashboard-jupyterlite/main/content/data/NC.csv"
LP_PATH = "https://raw.githubusercontent.com/DougRzz/nc-dashboard-jupyterlite/main/content/data/LP.csv"  
# GLOBAL_PATH = r"./data/global_data.json"
global_data = {
  "nc_price": 0.00018468120048354127,
  "uni_v2_price": 0.03922657593908290804,
  "circulating_supply": 41204856,
  "hodlers": 88,
  "total_liquidity": 83943.606229,
  "locked_lp_tokens": 62057.88,
  "reward": 1957000
}

pd.options.mode.chained_assignment = None


# def read_global_data():
#     # with open(GLOBAL_PATH, 'r') as f:
#     #     data = json.load(f)
#     data = global_data
#     return data


def get_all_addresses(df):
    addresses = set(df.From.unique())
    addresses.update(df.To.unique())
    addresses = list(addresses)
    addresses.append('')
    addresses.sort()
    return addresses


def process_transaction_data(df, address=None, methods=None, scale=1, invert=False, subtract=None):
    if address:
        trn = df[(df['From'] == address) | (df['To'] == address)]
    else:
        trn = df

    if methods:
        trn = trn[trn['Method'].isin(methods)]

    trn.loc[:, 'Balance'] = trn['Quantity']
    trn.loc[:, 'Balance'][trn.From == address] *= -1
    trn.loc[trn['Balance'] < 0, 'Balance'] /= scale
    trn.loc[:, 'Balance'] = trn['Balance'].cumsum()
    if invert:
        trn.loc[:, 'Balance'] *= -1
    if subtract:
        trn.loc[:, 'Balance'] = subtract - trn.loc[:, 'Balance']
    return trn


def get_days(trn):
    return (dt.datetime.today() - trn.DateTime.min()).days


def get_ratios(trn, address, methods=None):
    if methods is None:
        methods = []
    from_quantity = trn[trn.Method.isin(methods)][trn.From == address].Quantity.sum()
    to_quantity = trn[trn.Method.isin(methods)][trn.To == address].Quantity.sum()
    return from_quantity, to_quantity

nc = pd.read_csv(pyodide.open_url(NC_PATH), thousands=',', parse_dates=['DateTime'])
lp = pd.read_csv(pyodide.open_url(LP_PATH), thousands=',', parse_dates=['DateTime'])

# global_data = read_global_data()

nc_price = global_data['nc_price']
uni_v2_price = global_data['uni_v2_price']
circulating_supply = global_data['circulating_supply']
holders = global_data['hodlers']
total_liquidity = global_data['total_liquidity']
locked_lp_tokens = global_data['locked_lp_tokens']
reward = global_data['reward']

addresses = get_all_addresses(nc)

In [5]:
info_data= [
{'title': "Market Cap", 'value': f'${1_000_000_000 * nc_price:,.2f} USD'},
{'title': "Circulating supply", 'value': f'{circulating_supply:,} NC'},
{'title': "Circulating Market Cap", 'value': f'${circulating_supply * nc_price:,.2f} USD'},
{'title': "Holders", 'value': f'{holders} addresses'},

{'title': "Transactions", 'value': f'{len(nc)}'},
{'title': "Total liquidity", 'value': f'{total_liquidity:,.2f} UNI-V2'},
{'title': "LP value", 'value': f'{total_liquidity * uni_v2_price:,.2f} USD'},
{'title': "Locked LP tokens", 'value': f'{locked_lp_tokens:,.2f} UNI-V2'},

{'title': "Locked tokens ratio", 'value': f'{(locked_lp_tokens / total_liquidity) * 100:,.2f}%'},
{'title': "Locked LP value", 'value': f'{locked_lp_tokens * uni_v2_price:,.2f} USD'},
{'title': "Reward value", 'value': f'{(reward * nc_price):,.2f} USD'},
{'title': "Reward", 'value': f'{reward:,} NC'},
]

### Initialise ipyFlex layout

In [6]:

w = ipyflex.FlexLayout(
    style={"height": height}, #
    header={"title": "Natluk Community DASHBOAD", "style": {"backgroundColor": "rgb(53 53 53)"}}, # , "buttons":["export","import"]
    template_json=json.loads(pyodide.open_url("https://raw.githubusercontent.com/DougRzz/nc-dashboard-jupyterlite/main/content/NCw.json").getvalue()),
    editable=editable
)

In [7]:
style = ipw.HTML("""<style>.js-plotly-plot {height: 100%;}</style> """)


In [8]:
def create_plot(transactions, title = None):
    transactions = transactions.rename(columns={'DateTime': 'Date'})
    transactions.Date = transactions.Date.astype(str) # bug in jupyterlite, cant form a figwidget when datetime format x axis

    fig = px.line(transactions, x='Date', y='Balance', template = 'plotly_dark', title = f'<span style="font-size: 36px; font-family:calibri">{title}')
    fig.layout.xaxis.rangeslider.visible = False
    fig.layout.autosize = True      
    
    return go.FigureWidget(fig)

In [9]:


lp_transactions = process_transaction_data(lp, '0x0000000000000000000000000000000000000000', invert=True)

LP_fig_widget_main = create_plot(lp_transactions, title = "LP Balance history")


In [10]:
transactions = process_transaction_data(nc, '0xba93f37118f9d0ac9f620cd7bfb9fd79635db7d3', subtract=1_000_000_000)

CS_fig_widget = create_plot(transactions, title = "Circulating supply history")


In [11]:

from typing import Dict, List
def financial_info_factory(data: List[Dict], logo_url: str = None) -> v.Html:
    """
    Create info cards
    """
    children = []
    if logo_url is not None:
        logo = v.Card(
            outlined=True,
            class_='ma-1',
            children=[v.Img(src=logo_url, height='130px', contain=True)],
            style_='width: calc(12.5% - 8px); min-width: 140px; height: calc(50% - 8px)',
        )
        children.append(logo)

    for item in data:
        card = v.Card(
            outlined=True,
            class_='ma-1',
            children=[
                v.CardTitle(
                    primary_title=True,
                    children=[item['title']],
                    style_='min-height: 100px; font-size: 16px; color: #51ef98',
                ),
                v.CardText(children=[str(item['value'])], style_=' font-size: 12px'),
            ],
            style_=' width: calc(12.5% - 8px); min-width: 140px; height: calc(50% - 8px)', #height:100%:
        )
        children.append(card)
    return v.Row(
        tag='div',
        class_='d-flex align-content-start flex-wrap',
        children=children,
        # style_='flex-wrap: wrap',
    )

In [12]:
info = financial_info_factory(info_data, 'https://natluk.finance/assets/logo-NC-spinning-slow.4b211062.gif')

widgets = {
    "info": info,  # single values
    "Circulating supply": CS_fig_widget, # ticker  price chart
    "LP balance": LP_fig_widget_main, # price hitory
}

In [13]:
for key, value in widgets.items():
    w.add(key, value)

In [14]:
# def one_address_page(nc, lp, address, nc_price, uni_v2_price):

def onChange_addresses_select(change):
    if change['type'] == 'change' and change['name'] == 'v_model':
        addresses_select.loading = True
        address = change['new']
        transactions = process_transaction_data(nc, address, scale=0.95)
        has_transactions = len(transactions) > 0

        if has_transactions:
            addresses_select.rules = None
            lp_transactions = process_transaction_data(lp, address)
            farm_transactions = process_transaction_data(lp, address, FARMING_METHODS)
            has_lp_transactions = len(lp_transactions) > 0
            has_farm_transactions = len(farm_transactions) > 0
            balance = abs(transactions.tail(1)['Balance'].values[0])
            sells, buys = get_ratios(transactions, address, BUY_SELL_METHODS)

            # transactions, lp_transactions, farm_transactions, has_lp_transactions, has_farm_transactions,balance,sells, buys =  one_address_page(nc, lp, address, nc_price, uni_v2_price)

            transfered, received = get_ratios(transactions, address, ['Transfer'])

            NC_stats = [ 
                {'title':  "Time", 'value': f'{get_days(transactions)} days'},
                {'title':  "Balance", 'value': f'{balance:,.2f} NC'},
                {'title':  "Buys", 'value': f'{buys:,.2f} NC'},
                {'title':  "Sells", 'value': f'{sells:,.2f} NC'},
                {'title':  "Transfered", 'value': f'{transfered:,.2f} NC'},
                {'title':  "Received", 'value': f'{received:,.2f} NC'},
                {'title':  "NC price", 'value': f"{nc_price:,.9f} USD"},
                {'title':  "Value", 'value': f"{balance * nc_price:,.2f} USD"},
            ]

            NC_info = financial_info_factory(NC_stats, 'https://natluk.finance/assets/logo-NC-spinning-slow.4b211062.gif')

            fig_widget = w.children['NC history']
            title= "NC Balance History"
            fig_widget.data[0].x = transactions['DateTime'].astype(str)
            fig_widget.data[0].y = transactions['Balance']
            fig_widget.layout.xaxis.visible=True
            fig_widget.layout.yaxis.visible=True

            NC_col.children=[NC_info]
        #             NC_history_col.children=[NC_fig_widget]

            if has_lp_transactions:
                lp_balance = abs(lp_transactions.tail(1)['Balance'].values[0])
                removed, added = get_ratios(lp_transactions, address, LP_METHODS)
                LP_stats = [
                    {'title':  "Time", 'value': f'{get_days(lp_transactions)} days'},
                    {'title':  "Balance", 'value': f'{lp_balance:,.2f} UNI-V2'},
                    {'title':  "Added liquidity", 'value': f'{added:,.2f} UNI-V2'},
                    {'title':  "Removed liquidity", 'value': f'{removed:,.2f} UNI-V2'},
                    {'title':  "LP price", 'value': f'{uni_v2_price:,.9f} USD'},
                    {'title':  "Value", 'value': f'{(lp_balance * uni_v2_price):,.2f} USD'},
                ]
                LP_info = financial_info_factory(LP_stats, 'https://natluk.finance/assets/logo-NC-spinning-slow.4b211062.gif')


                LP_col.children=[LP_info]

                fig_widget = w.children['LP history']
                title= "LP Balance History"
                # fig_widget = plot_dict[title]        
                fig_widget.data[0].x = lp_transactions['DateTime'].astype(str)
                fig_widget.data[0].y = lp_transactions['Balance']
                fig_widget.layout.xaxis.visible=True
                fig_widget.layout.yaxis.visible=True

            else:
                LP_col.children=[]
                LP_history_col.children=[]


            if has_farm_transactions:
                farm_balance = abs(-farm_transactions.tail(1)['Balance'].values[0])
                added, removed = get_ratios(lp_transactions, address, FARMING_METHODS)
                _, claimed = get_ratios(transactions, address, ['Claim'])
                claimed_value = nc_price * claimed
                stacked_value = farm_balance * uni_v2_price

                Farm_stats = [
                    {'title':  "Time", 'value': f'{get_days(farm_transactions)} days'},
                    {'title':  "Balance", 'value': f'{farm_balance:,.2f} UNI-V2'},
                    {'title':  "Staked", 'value': f'{added:,.2f} UNI-V2'},
                    {'title':  "Unstaked", 'value': f'{removed:,.2f} UNI-V2'},
                    {'title':  "Claimed", 'value': f'{claimed:,.2f} NC'},
                    {'title':  "Claimed Value", 'value': f'{claimed_value:,.2f} USD'},
                    {'title':  "Stacked Value", 'value': f'{stacked_value:,.2f} USD'},
                ]

                farm_info = financial_info_factory(Farm_stats, 'https://natluk.finance/assets/logo-NC-spinning-slow.4b211062.gif')
                farm_col.children=[farm_info]
            else:
                farm_col.children=[]
        else:
            addresses_select.rules = ['Please enter a valid address']
            NC_col.children=[]
            # fig_widget = plot_dict["NC Balance History"]      
            fig_widget = w.children['NC history']
            fig_widget.layout.xaxis.visible=False
            fig_widget.layout.yaxis.visible=False   

            LP_col.children=[]
            fig_widget =  w.children['LP history']        
            fig_widget.layout.xaxis.visible=False
            fig_widget.layout.yaxis.visible=False              
            # LP_history_col.children=[]
            farm_col.children=[]

        addresses_select.loading = False
        
            

In [15]:
addresses_select = v.Combobox(
                          class_='pa-8 d-flex align-center ', 
#                           style_ = 'min-height:100%',
#                           multiple=True ,
#                           chips=True ,
#                           deletable_chips=True ,
#                           filter=True,
                          clearable = True,
                          items = addresses,
                          v_model=[],
                          label='Select address from list or enter new address:',
                          disabled=False)

In [22]:
addresses_select.observe(onChange_addresses_select)
# addresses_select

In [17]:
w.add("Select Address", addresses_select)

In [18]:
# addresses_select.v_model = '0x019ab42f50c89d8962bbcf33b94a39718a0058f5'

In [19]:
import pandas as pd
import numpy as np
NC_col = v.Col(children=[])      
w.add("NC stats", NC_col)
NC_plot = create_plot(pd.DataFrame( {'Date':[np.nan], 'Balance':[np.nan]}), title = "NC Balance History")
NC_plot.layout.xaxis.visible=False
NC_plot.layout.yaxis.visible=False
w.add("NC history", NC_plot)

LP_col = v.Col(children=[])      
w.add("LP stats", LP_col)
LP_plot = create_plot(pd.DataFrame( {'Date':[np.nan], 'Balance':[np.nan]}), title = "LP Balance History")
LP_plot.layout.xaxis.visible=False
LP_plot.layout.yaxis.visible=False
w.add("LP history", LP_plot)

farm_col = v.Col(children=[])
w.add("Farm stats", farm_col)


In [20]:
style = ipw.HTML("""<style>.js-plotly-plot {height: 100%;}</style> """)
ipw.VBox([w, style])

In [21]:
w.children['NC history']

FigureWidget({
    'data': [{'hovertemplate': 'Date=%{x}<br>Balance=%{y}<extra></extra>',
              'legen…